In [2]:
import duckdb

In [3]:
duckdb.__version__

'1.2.1'

In [4]:
result = duckdb.sql('SELECT 3')
result.show()

┌───────┐
│   3   │
│ int32 │
├───────┤
│     3 │
└───────┘



In [5]:
con = duckdb.connect(database=':memory:')
con.execute('CREATE TABLE test (a INTEGER, b VARCHAR)')
con.execute("INSERT INTO test VALUES (1, 'foo')")
con.execute("INSERT INTO test VALUES (2, 'bar')")
result = con.sql("SELECT * FROM test")
result.show()

┌───────┬─────────┐
│   a   │    b    │
│ int32 │ varchar │
├───────┼─────────┤
│     1 │ foo     │
│     2 │ bar     │
└───────┴─────────┘



In [6]:
con.execute("INSTALL httpfs")

In [7]:
con.execute("LOAD httpfs")

In [8]:
population = con.read_csv("https://bit.ly/3KoiZR0")

In [9]:
type(population)

duckdb.duckdb.DuckDBPyRelation

In [10]:
con.execute("SELECT * from population limit 2").fetchall()

[('Afghanistan ',
  'ASIA (EX. NEAR EAST)         ',
  31056997,
  647500,
  '48,0',
  '0,00',
  '23,06',
  '163,07',
  700,
  '36,0',
  '3,2',
  '12,13',
  '0,22',
  '87,65',
  '1',
  '46,6',
  '20,34',
  '0,38',
  '0,24',
  '0,38'),
 ('Albania ',
  'EASTERN EUROPE                     ',
  3581655,
  28748,
  '124,6',
  '1,26',
  '-4,93',
  '21,52',
  4500,
  '86,5',
  '71,2',
  '21,09',
  '4,42',
  '74,49',
  '3',
  '15,11',
  '5,22',
  '0,232',
  '0,188',
  '0,579')]

In [11]:
population.count("*").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          227 │
└──────────────┘



In [12]:
population_table = con.table("population")

In [14]:
population_table.show()

┌────────────────────┬─────────────────────────────────────┬────────────┬────────────────┬────────────────────────────┬──────────────────────────────┬───────────────┬────────────────────────────────────┬────────────────────┬──────────────┬───────────────────┬────────────┬───────────┬───────────┬─────────┬───────────┬───────────┬─────────────┬──────────┬─────────┐
│      Country       │               Region                │ Population │ Area (sq. mi.) │ Pop. Density (per sq. mi.) │ Coastline (coast/area ratio) │ Net migration │ Infant mortality (per 1000 births) │ GDP ($ per capita) │ Literacy (%) │ Phones (per 1000) │ Arable (%) │ Crops (%) │ Other (%) │ Climate │ Birthrate │ Deathrate │ Agriculture │ Industry │ Service │
│      varchar       │               varchar               │   int64    │     int64      │          varchar           │           varchar            │    varchar    │              varchar               │       int64        │   varchar    │      varchar      │  varchar

In [26]:
population_table.aggregate("Region, sum(Population) as total_pop").order("total_pop DESC").show()

┌─────────────────────────────────────┬────────────┐
│               Region                │ total_pop  │
│               varchar               │   int128   │
├─────────────────────────────────────┼────────────┤
│ ASIA (EX. NEAR EAST)                │ 3687982236 │
│ SUB-SAHARAN AFRICA                  │  749437000 │
│ LATIN AMER. & CARIB                 │  561824599 │
│ WESTERN EUROPE                      │  396339998 │
│ NORTHERN AMERICA                    │  331672307 │
│ C.W. OF IND. STATES                 │  280081548 │
│ NEAR EAST                           │  195068377 │
│ NORTHERN AFRICA                     │  161407133 │
│ EASTERN EUROPE                      │  119914717 │
│ OCEANIA                             │   33131662 │
│ BALTICS                             │    7184974 │
├─────────────────────────────────────┴────────────┤
│ 11 rows                                2 columns │
└──────────────────────────────────────────────────┘



In [23]:
population_table.filter("Population > 100000000").project("Country", "Population").show()

┌────────────────┬────────────┐
│    Country     │ Population │
│    varchar     │   int64    │
├────────────────┼────────────┤
│ Bangladesh     │  147365352 │
│ Brazil         │  188078227 │
│ China          │ 1313973713 │
│ India          │ 1095351995 │
│ Indonesia      │  245452739 │
│ Japan          │  127463611 │
│ Mexico         │  107449525 │
│ Nigeria        │  131859731 │
│ Pakistan       │  165803560 │
│ Russia         │  142893540 │
│ United States  │  298444215 │
├────────────────┴────────────┤
│ 11 rows           2 columns │
└─────────────────────────────┘



In [24]:
over_100m = population_table.filter("Population > 100000000")

In [28]:
over_100m.filter('"GDP ($ per capita)" > 10000').show()

┌────────────────┬─────────────────────────────────────┬────────────┬────────────────┬────────────────────────────┬──────────────────────────────┬───────────────┬────────────────────────────────────┬────────────────────┬──────────────┬───────────────────┬────────────┬───────────┬───────────┬─────────┬───────────┬───────────┬─────────────┬──────────┬─────────┐
│    Country     │               Region                │ Population │ Area (sq. mi.) │ Pop. Density (per sq. mi.) │ Coastline (coast/area ratio) │ Net migration │ Infant mortality (per 1000 births) │ GDP ($ per capita) │ Literacy (%) │ Phones (per 1000) │ Arable (%) │ Crops (%) │ Other (%) │ Climate │ Birthrate │ Deathrate │ Agriculture │ Industry │ Service │
│    varchar     │               varchar               │   int64    │     int64      │          varchar           │           varchar            │    varchar    │              varchar               │       int64        │   varchar    │      varchar      │  varchar   │  varcha

In [29]:
over_100m.filter('"GDP ($ per capita)" > 10000').count("*").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            2 │
└──────────────┘



In [32]:
population_table.except_(over_100m).aggregate("Region, sum(Population) as total_pop, count(*)").order("total_pop DESC").show()

┌─────────────────────────────────────┬───────────┬──────────────┐
│               Region                │ total_pop │ count_star() │
│               varchar               │  int128   │    int64     │
├─────────────────────────────────────┼───────────┼──────────────┤
│ SUB-SAHARAN AFRICA                  │ 617577269 │           50 │
│ ASIA (EX. NEAR EAST)                │ 592571266 │           22 │
│ WESTERN EUROPE                      │ 396339998 │           28 │
│ LATIN AMER. & CARIB                 │ 266296847 │           43 │
│ NEAR EAST                           │ 195068377 │           16 │
│ NORTHERN AFRICA                     │ 161407133 │            6 │
│ C.W. OF IND. STATES                 │ 137188008 │           11 │
│ EASTERN EUROPE                      │ 119914717 │           12 │
│ NORTHERN AMERICA                    │  33228092 │            4 │
│ OCEANIA                             │  33131662 │           21 │
│ BALTICS                             │   7184974 │           

In [33]:
africa = population_table.filter("Region ~ '.*AFRICA.*'")

In [34]:
africa.intersect(over_100m).show()

┌──────────┬─────────────────────────────────────┬────────────┬────────────────┬────────────────────────────┬──────────────────────────────┬───────────────┬────────────────────────────────────┬────────────────────┬──────────────┬───────────────────┬────────────┬───────────┬───────────┬─────────┬───────────┬───────────┬─────────────┬──────────┬─────────┐
│ Country  │               Region                │ Population │ Area (sq. mi.) │ Pop. Density (per sq. mi.) │ Coastline (coast/area ratio) │ Net migration │ Infant mortality (per 1000 births) │ GDP ($ per capita) │ Literacy (%) │ Phones (per 1000) │ Arable (%) │ Crops (%) │ Other (%) │ Climate │ Birthrate │ Deathrate │ Agriculture │ Industry │ Service │
│ varchar  │               varchar               │   int64    │     int64      │          varchar           │           varchar            │    varchar    │              varchar               │       int64        │   varchar    │      varchar      │  varchar   │  varchar  │  varchar  │ v